# Movies Recommender System

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


![](http://labs.criteo.com/wp-content/uploads/2017/08/CustomersWhoBought3.jpg)

In [3]:
!unzip /content/drive/MyDrive/Recsystem/movies_metadata.csv.zip

Archive:  /content/drive/MyDrive/Recsystem/movies_metadata.csv.zip
  inflating: movies_metadata.csv     


In [4]:
%matplotlib inline
import pandas as pd
import numpy as np


import warnings; warnings.simplefilter('ignore')

In [5]:
movie_filter = pd.read_csv('/content/drive/MyDrive/Recsystem/links_small.csv')
movie_filter = movie_filter[movie_filter['tmdbId'].notnull()]['tmdbId'].astype('int')

In [6]:
movies = pd.read_csv('/content/movies_metadata.csv')
movies.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [7]:
movies.shape

(45466, 24)

In [8]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

In [9]:
movies['revenue'].value_counts(ascending=False)

0.0           38052
12000000.0       20
11000000.0       19
10000000.0       19
2000000.0        18
              ...  
36565280.0        1
439564.0          1
35610100.0        1
10217873.0        1
1413000.0         1
Name: revenue, Length: 6863, dtype: int64

In [10]:
movies['budget'] = pd.to_numeric(movies['budget'], errors='coerce')
movies['budget'].value_counts(ascending=False)

0.0            36573
5000000.0        286
10000000.0       259
20000000.0       243
2000000.0        242
               ...  
270000000.0        1
923.0              1
72500000.0         1
2160000.0          1
1254040.0          1
Name: budget, Length: 1223, dtype: int64

In [11]:
movies['adult'].value_counts()

False                                                                                                                             45454
True                                                                                                                                  9
 - Written by Ørnås                                                                                                                   1
 Rune Balot goes to a casino connected to the October corporation to try to wrap up her case once and for all.                        1
 Avalanche Sharks tells the story of a bikini contest that turns into a horrifying affair when it is hit by a shark avalanche.        1
Name: adult, dtype: int64

In [12]:
movies.drop('adult', axis=1)

,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000.0,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,NaN,65000000.0,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0.0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,NaN,16000000.0,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.859495,...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,"{'id': 96871, 'name': 'Father of the Bride Col...",0.0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.387519,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45461,NaN,0.0,"[{'id': 18, 'name': 'Drama'}, {'id': 10751, 'n...",http://www.imdb.com/title/tt6209470/,439050,tt6209470,fa,رگ خواب,Rising and falling between a man and woman.,0.072051,...,NaN,0.0,90.0,"[{'iso_639_1': 'fa', 'name': 'فارسی'}]",Released,Rising and falling between a man and woman,Subdue,False,4.0,1.0
45462,NaN,0.0,"[{'id': 18, 'name': 'Drama'}]",NaN,111109,tt2028550,tl,Siglo ng Pagluluwal,An artist struggles to finish his work while a...,0.178241,...,2011-11-17,0.0,360.0,"[{'iso_639_1': 'tl', 'name': ''}]",Released,NaN,Century of Birthing,False,9.0,3.0
45463,NaN,0.0,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",NaN,67758,tt0303758,en,Betrayal,"When one of her hits goes wrong, a professiona...",0.903007,...,2003-08-01,0.0,90.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,A deadly game of wits.,Betrayal,False,3.8,6.0
45464,NaN,0.0,[],NaN,227506,tt0008536,en,Satana likuyushchiy,"In a small town live two brothers, one a minis...",0.003503,...,1917-10-21,0.0,87.0,[],Released,NaN,Satan Triumphant,False,0.0,0.0


In [13]:
from ast import literal_eval

def extract_franchise_name(data):
  if isinstance(data, dict):
    return data['name']
  else:
    return np.nan

movie_franchises = movies[movies['belongs_to_collection'].notnull()]
movie_franchises['belongs_to_collection'] = movie_franchises['belongs_to_collection'].apply(literal_eval).apply(extract_franchise_name)
movie_franchises = movie_franchises[movie_franchises['belongs_to_collection'].notnull()]

In [14]:
revenues_by_franchise = movie_franchises.pivot_table(index='belongs_to_collection', values='revenue', aggfunc={'revenue': ['mean', 'sum', 'count']}).reset_index()
revenues_by_franchise.sort_values('sum', ascending=False).head(10)

,belongs_to_collection,count,mean,sum
552,Harry Potter Collection,8,9.634209e+08,7.707367e+09
1160,Star Wars Collection,8,9.293118e+08,7.434495e+09
646,James Bond Collection,26,2.733450e+08,7.106970e+09
1317,The Fast and the Furious Collection,8,6.406373e+08,5.125099e+09
968,Pirates of the Caribbean Collection,5,9.043154e+08,4.521577e+09
1550,Transformers Collection,5,8.732202e+08,4.366101e+09
325,Despicable Me Collection,6,6.151784e+08,3.691070e+09
1491,The Twilight Collection,5,6.684215e+08,3.342107e+09
610,Ice Age Collection,5,6.433417e+08,3.216709e+09
666,Jurassic Park Collection,4,7.578710e+08,3.031484e+09


In [15]:
budget_by_franchise = movie_franchises.pivot_table(index='belongs_to_collection', values='budget', aggfunc={'budget': ['mean', 'sum', 'count']}).reset_index()
budget_by_franchise.sort_values('sum', ascending=False).head(10)

,belongs_to_collection,count,mean,sum
646,James Bond Collection,26,5.921731e+07,1.539650e+09
552,Harry Potter Collection,8,1.600000e+08,1.280000e+09
968,Pirates of the Caribbean Collection,5,2.500000e+08,1.250000e+09
1317,The Fast and the Furious Collection,8,1.261250e+08,1.009000e+09
1656,X-Men Collection,6,1.638333e+08,9.830000e+08
1550,Transformers Collection,5,1.930000e+08,9.650000e+08
1160,Star Wars Collection,8,1.067938e+08,8.543500e+08
1352,The Hobbit Collection,3,2.500000e+08,7.500000e+08
1483,The Terminator Collection,5,1.322800e+08,6.614000e+08
835,Mission: Impossible Collection,5,1.300000e+08,6.500000e+08


In [16]:
invalid_indices = movies[movies['imdb_id'] == '0'].index
movies = movies.drop(invalid_indices, axis=0)
movies['id'] = movies['id'].astype('int')

In [17]:
movies = movies[movies['id'].isin(movie_filter)]
list(movies.columns.values)

['adult',
 'belongs_to_collection',
 'budget',
 'genres',
 'homepage',
 'id',
 'imdb_id',
 'original_language',
 'original_title',
 'overview',
 'popularity',
 'poster_path',
 'production_companies',
 'production_countries',
 'release_date',
 'revenue',
 'runtime',
 'spoken_languages',
 'status',
 'tagline',
 'title',
 'video',
 'vote_average',
 'vote_count']

In [18]:
movies['tagline'] = movies['tagline'].fillna('')
movies['overview'] = movies['overview'].fillna('')
movies['description'] = movies[['tagline', 'overview']].agg(' '.join, axis=1)

In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer

tf_idf_vectorizer = TfidfVectorizer(stop_words='english')
tf_idf_matrix = tf_idf_vectorizer.fit_transform(movies['description'])

In [20]:
tf_idf_matrix.shape

(9099, 30570)

In [21]:
from sklearn.metrics.pairwise import linear_kernel

cos_similarity = linear_kernel(tf_idf_matrix, tf_idf_matrix)
cos_similarity.shape

(9099, 9099)

In [22]:
cos_similarity[0:5]

array([[1.        , 0.01667385, 0.        , ..., 0.        , 0.00671988,
        0.        ],
       [0.01667385, 1.        , 0.04322203, ..., 0.00892538, 0.01697785,
        0.        ],
       [0.        , 0.04322203, 1.        , ..., 0.        , 0.00643161,
        0.01237912],
       [0.        , 0.        , 0.        , ..., 0.        , 0.01917227,
        0.        ],
       [0.        , 0.00649646, 0.02310156, ..., 0.01132077, 0.00320429,
        0.        ]])

In [23]:
movies = movies.reset_index()

In [24]:
from operator import itemgetter

def recommend_movies_on_similarity(movies, title, movie_count):
    movie_indices = recommend_indices_on_similarity(movies, title, movie_count)
    return movies.iloc[movie_indices]['title']

def recommend_indices_on_similarity(movies, title, movie_count):
    idx = movies[movies['title'] == title].index[0]
    cos_similarity_title = list(enumerate(cos_similarity[idx]))
    cos_similarity_title = sorted(cos_similarity_title, key=itemgetter(1), reverse=True)
    movie_indices = [i[0] for i in cos_similarity_title[1:movie_count]]
    return movie_indices

In [25]:
recommend_movies_on_similarity(movies, 'Fury', 10)

3981                  Gloria
3282              Hiding Out
2340           Hang 'em High
5314            Mean Streets
1586    Jane Austen's Mafia!
2879           Things Change
8107                  Looper
3312             The Mexican
220               Houseguest
Name: title, dtype: object

In [26]:
recommend_movies_on_similarity(movies, 'Ice Age', 10)

7246    Ice Age: Dawn of the Dinosaurs
6367             Ice Age: The Meltdown
8067        Ice Age: Continental Drift
6884                         10,000 BC
5440                      Two Brothers
8588                They Came Together
9053    Ice Age: The Great Egg-Scapade
680                       House Arrest
6654                   Blades of Glory
Name: title, dtype: object

In [27]:
def clean_and_get_int_col_data(df, col_name):
    return df[df[col_name].notnull()][col_name].astype('int')


In [28]:
def convert_col_to_int(df, col_name):
    df[col_name] = df[col_name].astype('int')

In [29]:
def filter_well_rate_movies(movies, min_vote_count):
    filtered_movies = movies[(movies['vote_count'] >= min_vote_count) & (movies['vote_count'].notnull()) & (movies['vote_average'].notnull())]
    convert_col_to_int(filtered_movies, 'vote_count')
    convert_col_to_int(filtered_movies, 'vote_average')
    return filtered_movies

In [30]:
def recommend_movies_on_similarity_and_rating(movies, title, movie_count):
    movie_indices = recommend_indices_on_similarity(movies, title, 50)
    
    movies_trimmed = movies.iloc[movie_indices][['title', 'vote_count', 'vote_average']]
    vote_count_int = clean_and_get_int_col_data(movies_trimmed, 'vote_count')
    vote_avg_int = clean_and_get_int_col_data(movies_trimmed, 'vote_average')
    C = vote_avg_int.mean()
    m = vote_count_int.quantile(0.65)
    filtered_movies = filter_well_rate_movies(movies_trimmed, m)
    filtered_movies['wr'] = filtered_movies.apply(lambda x: (x['vote_count']/(x['vote_count']+m) * x['vote_average']) + (m/(m+x['vote_count']) * C), axis=1)
    filtered_movies = filtered_movies.sort_values('wr', ascending=False).head(movie_count)
    return filtered_movies

In [36]:
recommend_movies_on_similarity_and_rating(movies, 'House Arrest', 10)

,title,vote_count,vote_average,wr
3995,Ice Age,3954,7,6.952311
8582,Boyhood,2008,7,6.909079
7797,Submarine,452,7,6.669537
6683,This Is England,370,7,6.616258
3692,American Pie 2,1375,6,5.962625
677,Matilda,892,6,5.945108
1617,The Parent Trap,768,6,5.937600
7043,Zack and Miri Make a Porno,734,6,5.935169
1699,Sixteen Candles,455,6,5.904698
7394,It's Complicated,370,6,5.888770


In [37]:


recommend_movies_on_similarity_and_rating(movies, 'Toy Story', 10)

,title,vote_count,vote_average,wr
284,The Shawshank Redemption,8358,8,7.945662
3806,Harry Potter and the Philosopher's Stone,7188,7,6.966725
7535,Toy Story 3,4710,7,6.949998
2502,Toy Story 2,3914,7,6.940369
6429,The Devil Wears Prada,3198,7,6.927876
6290,Match Point,1134,7,6.817901
1959,The Fly,1038,7,6.804000
1933,The Thin Red Line,793,7,6.756577
994,Manhattan,600,7,6.699256
889,Rebel Without a Cause,351,7,6.568016
